In [19]:
import pandas as pd

df = pd.read_csv('fema_disasters_1994_2020.csv')
# Cast data types
df = df.astype({
    'fipsStateCode': 'object',
    'fipsCountyCode': 'object'
})

# Show columns and data types
df.dtypes

femaDeclarationString        object
disasterNumber              float64
state                        object
declarationType              object
declarationDate              object
fyDeclared                  float64
incidentType                 object
declarationTitle             object
ihProgramDeclared           float64
iaProgramDeclared           float64
paProgramDeclared           float64
hmProgramDeclared           float64
Date                         object
incidentBeginDate            object
incidentEndDate              object
disasterCloseoutDate         object
fipsStateCode                object
fipsCountyCode               object
placeCode                   float64
designatedArea               object
declarationRequestNumber    float64
hash                         object
lastRefresh                  object
id                           object
dtype: object

In [20]:
# Columns to keep:
renamed_columns = {
    'femaDeclarationString': 'disaster_id',
    'state': 'disaster_state',
    'designatedArea': 'disaster_county',
    'fipsStateCode': 'disaster_state_code',
    'fipsCountyCode': 'disaster_county_code',
    'declarationType': 'disaster_declaration',
    'declarationTitle': 'disaster_name',
    'incidentType': 'disaster_type',
    'incidentBeginDate': 'disaster_date'
}

# Filter out unused columns
results = df[renamed_columns.keys()].copy()

# Rename kept columns
results.rename(columns=renamed_columns, inplace=True)

# Define initial filters
is_fl = results['disaster_state'] == 'FL'
is_county = ~(results['disaster_county_code'] == 0)

# Apply initial filters
results = results[is_fl & is_county]

results.head(10)

,disaster_id,disaster_state,disaster_county,disaster_state_code,disaster_county_code,disaster_declaration,disaster_name,disaster_type,disaster_date
679,DR-209-FL,FL,Broward (County),12,11,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
688,DR-209-FL,FL,St. Lucie (County),12,111,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
691,DR-209-FL,FL,Palm Beach (County),12,99,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
696,DR-209-FL,FL,Martin (County),12,85,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
703,DR-209-FL,FL,Glades (County),12,43,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
735,DR-209-FL,FL,Dade (County),12,86,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
748,DR-209-FL,FL,Lee (County),12,71,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
860,DR-209-FL,FL,Collier (County),12,21,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
867,DR-209-FL,FL,Hendry (County),12,51,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
869,DR-209-FL,FL,Monroe (County),12,87,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z


In [21]:
# Convert to string, decode to remove encoded characters, then strip the '.0' at the end.
results['disaster_county_code'] = results['disaster_county_code'].astype('|S').str.decode('utf-8').str.strip('.0')
results['disaster_state_code'] = results['disaster_state_code'].astype('|S').str.decode('utf-8').str.strip('.0')

# Check if county code is only two digits. If it is, add a '0' to the beginning of the code.
# Documentation on fips codes: https://en.wikipedia.org/wiki/FIPS_county_code
#under_fips_length = results['disaster_county_code'].str.len() < 3
results['disaster_county_code'] = results['disaster_county_code'].str.pad(width=3, side='left', fillchar='0')

results.head(10)

# -- Testing for disaster_county_code to find any values less than 3 characters long:
# results[results['disaster_county_code'].str.len() < 3].head(10)

,disaster_id,disaster_state,disaster_county,disaster_state_code,disaster_county_code,disaster_declaration,disaster_name,disaster_type,disaster_date
679,DR-209-FL,FL,Broward (County),12,011,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
688,DR-209-FL,FL,St. Lucie (County),12,111,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
691,DR-209-FL,FL,Palm Beach (County),12,099,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
696,DR-209-FL,FL,Martin (County),12,085,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
703,DR-209-FL,FL,Glades (County),12,043,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
735,DR-209-FL,FL,Dade (County),12,086,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
748,DR-209-FL,FL,Lee (County),12,071,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
860,DR-209-FL,FL,Collier (County),12,021,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
867,DR-209-FL,FL,Hendry (County),12,051,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z
869,DR-209-FL,FL,Monroe (County),12,087,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z


In [22]:
# Create an extra column that combines the fips state and county codes. This is how they will be joined with birth
# totals from the CDC.
results['disaster_fips_code'] = results['disaster_state_code'] + results['disaster_county_code']

results.head(10)

,disaster_id,disaster_state,disaster_county,disaster_state_code,disaster_county_code,disaster_declaration,disaster_name,disaster_type,disaster_date,disaster_fips_code
679,DR-209-FL,FL,Broward (County),12,011,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12011
688,DR-209-FL,FL,St. Lucie (County),12,111,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12111
691,DR-209-FL,FL,Palm Beach (County),12,099,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12099
696,DR-209-FL,FL,Martin (County),12,085,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12085
703,DR-209-FL,FL,Glades (County),12,043,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12043
735,DR-209-FL,FL,Dade (County),12,086,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12086
748,DR-209-FL,FL,Lee (County),12,071,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12071
860,DR-209-FL,FL,Collier (County),12,021,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12021
867,DR-209-FL,FL,Hendry (County),12,051,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12051
869,DR-209-FL,FL,Monroe (County),12,087,DR,HURRICANE BETSY,Hurricane,1965-09-14T04:00:00.000Z,12087


In [23]:
# Export to CSV
results.to_csv('fema_fl_disasters_1994_2020.csv', index=False)